In [1]:
import pandas as pd
import torch
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# !pip install pytorch_lightning

In [4]:
import pytorch_lightning as pl
# import lightning as L

In [5]:
!pwd

/content


In [6]:
%cd drive/MyDrive/FSLMal

/content/drive/MyDrive/FSLMal


In [7]:
!ls

CustomDataset.py  data_labelFSL.csv  logs		     __pycache__
Data		  Data.zip	     MalClassifierForFSL.py  TrainCNNForFSL.ipynb


In [8]:
from MalClassifierForFSL import MalClassifierForFSL

In [9]:
hyper_parameter = {
    'max_length': 400,
    'lr': 1e-6,  # Starting Learning Rate
    'epochs' : 5,
    'optimizer' : 'adamw',
    'gamma' : 0.9,
    'batch_size': 32,
    'DataFrame': pd.read_csv("./data_labelFSL.csv"),
    "class_num": 9,
    "static_path": "./Data/Data/img/",
    "dynamic_path": "./Data/Data/tfidf",
    "img_type": 'orig'
}

In [10]:
model = MalClassifierForFSL(hyper_parameter)

In [11]:
np.random.seed(42)
torch.manual_seed(42)

In [12]:
print(torch.cuda.is_available())

False


In [13]:
print(torch.cuda.device_count())

0


In [14]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [15]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [16]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger(save_dir='logs')

In [17]:
model = model.to(device)

In [18]:
trainer = pl.Trainer(
    # callbacks=[checkpoint_callback],
    max_epochs=model.EPOCHS,
    deterministic=torch.cuda.is_available(),
    logger=logger,
    # devices=[0] if torch.cuda.is_available() else None,
    #tpu_cores=1
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model=model)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:377: Found unsupported keys in the optimizer configuration: {'scheduler'}
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | CNN_STATIC   | Sequential       | 166 K  | train
1 | LSTM_DYNAMIC | Sequential       | 848 K  | train
2 | relu         | ReLU             | 0      | train
3 | drop         | Dropout          | 0      | train
4 | d2           | Linear           | 1.1 M  | train
5 | d3           | Linear           | 32.9 K | train
6 | d4           | Linear           | 8.3 K  | train
7 | d5           | Linear           | 585    | train
8 | softmax      | Softmax          | 0      | train
9 | lossF        | CrossEntropyLoss | 0      | train
----------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.798     Total esti

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# model.train_dataloader()

In [ ]:
trainer.test(model)

In [ ]:
model.train_outputs_epoch

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=logs